## MultiIndex: uma forma de armazenar informações multidimensionais em um vetor
---

In [1]:
import numpy as np
import pandas as pd

pandas tem o tipo `MultiIndex` capaz de receber, como índice, tuplas contendo diversos valores, sendo, assim, possível armazenar mais de uma informação em um `Series`.

para isto, é antes necessário já dispor da lista de tuplas que servirá como índice mais tarde:

In [2]:
índices = [('b', 1), ('b', 0), ('c', 0), ('d', 4), ('d', 1), ('e', 0)]

em seguida, os valores e a `Series` devem ser declaradas normalmente:

In [3]:
vlr = [np.e, np.pi, np.pi**np.e, np.e/np.pi, -1/np.pi, np.sqrt(np.e)]

tudo = pd.Series(vlr, index=índices)
tudo

(b, 1)     2.718282
(b, 0)     3.141593
(c, 0)    22.459158
(d, 4)     0.865256
(d, 1)    -0.318310
(e, 0)     1.648721
dtype: float64

até agora, o que se tem é apenas tuplas em forma de índices, mas que, ainda, não é capaz de fazer o que `MultiIndex` faz. Para tal, é preciso redeclarar os índices para que este passe a ser do tipo `MultiIndex`:

In [4]:
índices = pd.MultiIndex.from_tuples(índices)

e, agora, é necessário reorganizar o vetor `tudo` com o método `.reindex()` que recebe como parâmetro a lista de tuplas:

In [5]:
tudo = tudo.reindex(índices)

agora, sim:

In [6]:
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   1    -0.318310
e  0     1.648721
dtype: float64

observe que o pandas organiza cada valor dos índices em hierarquias, onde cada espaço vazio corresponde ao valor na linha preenchida mais próxima.

mas, nem sempre é necessário criar um `MultiIndex` explicitamente: só em passar, através do parâmetro `index=`, uma lista contendo os arrays para construir os índices, o pandas já faz isso altomaticamente:

In [7]:
tudo = pd.Series(vlr, index=[['b', 'b', 'c', 'd', 'd', 'e'], [1, 0, 0, 4, 1, 0]])
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   1    -0.318310
e  0     1.648721
dtype: float64

ou mesmo, através de dicionários, onde, da mesma forma, os índices devem já ser os conjunto de arrays:

In [8]:
tudo = pd.Series(vlr, index={('b', 1): vlr[0], ('b', 0): vlr[1],
                             ('c', 0): vlr[2], ('d', 4): vlr[3],
                             ('d', 0): vlr[4], ('e', 0): vlr[5]})
tudo

b  1     2.718282
   0     3.141593
c  0    22.459158
d  4     0.865256
   0    -0.318310
e  0     1.648721
dtype: float64

este tipo de series pode ainda ser criado com os métodos `.from_arrays()`, `.from_frame()` e `.from_product()`, ou diretamente pela classe `.MultiIndex(levels=[<listas_indices>], labels=[<lista_valores>])`

especificamente, o método `.from_product()` funciona da seguinte maneira:
> pd.MultiIndex.from_product([[vl1, vl2, vl3], [vl4, vl5]])

haverá uma multiplicação de vetores aqui, resultando no níveis:
> {vl1: (vl4, vl5), vl2: (vl4, vl5), vl3: (vl4, vl5)}

#### ordenação
---

é interessante observar que o `MultiIndex` não funciona se os índices não estiverem organizados:

In [42]:
índices = [('b', 1), ('c', 0), ('b', 0), ('d', 1), ('e', 0), ('d', 4)]
vlr = [np.e, np.pi, np.pi**np.e, np.e/np.pi, -1/np.pi, np.sqrt(np.e)]
tudo = pd.Series(vlr, index=índices)

índices = pd.MultiIndex.from_tuples(índices)
tudo = tudo.reindex(índices)
tudo

b  1     2.718282
c  0     3.141593
b  0    22.459158
d  1     0.865256
e  0    -0.318310
d  4     1.648721
dtype: float64

para não precisar afzer isto manualmente, pode-se usar as funções `sort_index()` ou `sortlevel()`:

In [43]:
tudo = tudo.sort_index()
tudo

b  0    22.459158
   1     2.718282
c  0     3.141593
d  1     0.865256
   4     1.648721
e  0    -0.318310
dtype: float64

#### indexing e slicing em series
---

funciona da mesma forma que para pandas:

In [9]:
tudo['b']

1    2.718282
0    3.141593
dtype: float64

In [10]:
tudo[:, 0]

b     3.141593
c    22.459158
d    -0.318310
e     1.648721
dtype: float64

este slicing funciona da seguinte forma:
1. como tem `:`, isto significa que deve percorrer todos os primeiros valores dos índices;
2. assim, checa se estes têm o valor 0 (zero) na segunda posição;
3. se tiver, retorna o valor passado como para a series.

In [11]:
tudo['d', 4]

0.8652559794322651

In [12]:
tudo['b'][1] # ou tudo['b', 1]

2.718281828459045

In [13]:
tudo['b':'c']

b  1     2.718282
   0     3.141593
c  0    22.459158
dtype: float64

In [14]:
tudo[['b', 'c']]

b  1     2.718282
   0     3.141593
c  0    22.459158
dtype: float64

fancing indexing também funciona aqui:

In [15]:
tudo[tudo < 0]

d  0   -0.31831
dtype: float64

In [16]:
tudo[tudo > 2.8]

b  0     3.141593
c  0    22.459158
dtype: float64

#### em dataframmes
---

é importante ver que isto também funciona com dataframes:

In [17]:
tudo_df = pd.DataFrame({'tudo': tudo})
tudo_df

tudo
b 1   2.718282
  0   3.141593
c 0  22.459158
d 4   0.865256
  0  -0.318310
e 0   1.648721

os datafraes, desta forma, conseguem comportar muito mais informações, ficando cada vez mais semelhante a uma planilha excel, por exemplo.

é até mesmo possível adicionar informações a este dataframe:

In [18]:
tudo_df['adicionado'] = [1, 2, 3, 4, 5, 6]
tudo_df

tudo  adicionado
b 1   2.718282           1
  0   3.141593           2
c 0  22.459158           3
d 4   0.865256           4
  0  -0.318310           5
e 0   1.648721           6

#### .unstack() e .stack()
---

este método retorna a series de índices multidimensionais como um dataframe:

In [22]:
tudo_df = tudo.unstack()
tudo_df

,0,1,4
b,3.141593,2.718282,NaN
c,22.459158,NaN,NaN
d,-0.318310,NaN,0.865256
e,1.648721,NaN,NaN


o método `unstack()` ainda recebe o parâmetro `level=`, que recebe o número do nível ao qual deseja-se usar:

In [44]:
tudo_df_0 = tudo.unstack(level=0)
tudo_df_1 = tudo.unstack(level=1)

In [45]:
tudo_df_0

,b,c,d,e
0,22.459158,3.141593,NaN,-0.31831
1,2.718282,NaN,0.865256,NaN
4,NaN,NaN,1.648721,NaN


In [46]:
tudo_df_1

,0,1,4
b,22.459158,2.718282,NaN
c,3.141593,NaN,NaN
d,NaN,0.865256,1.648721
e,-0.318310,NaN,NaN


o método `.stack()` faz o contrário, transformando um dataframe em uma series de índices multidimensionais

In [23]:
tudo = tudo_df.stack()
tudo

b  0     3.141593
   1     2.718282
c  0    22.459158
d  0    -0.318310
   4     0.865256
e  0     1.648721
dtype: float64

#### nomeando os níveis
---

pode ser interessante dar nome aos níveis. isto é possível com o atributo `.index.names` que recebe um array com os nomes de cada nível:

In [24]:
tudo.index.names = ['letras', 'valores']
tudo

letras  valores
b       0           3.141593
        1           2.718282
c       0          22.459158
d       0          -0.318310
        4           0.865256
e       0           1.648721
dtype: float64

ou, pode já passar como parâmetro em `MultiIndex(names=[<lista_nomes>])`

#### em colunas
---

é importante ressaltar que estes níveis também podem ser passados para as colunas, nos dataframes:

In [25]:
col = pd.MultiIndex.from_product([['ontem', 'hoje', 'amanhã'], ['a', 'b']], names=['dia', 'ocorrência'])

df = pd.DataFrame(np.random.random((6, 6)), columns=col)
df

dia            ontem                hoje              amanhã          
ocorrência         a         b         a         b         a         b
0           0.628870  0.480733  0.994014  0.431046  0.687553  0.250792
1           0.409085  0.912318  0.929658  0.860127  0.636363  0.202969
2           0.506160  0.128801  0.439149  0.562535  0.778278  0.541486
3           0.627495  0.292769  0.986260  0.526518  0.975170  0.244955
4           0.023356  0.744610  0.251913  0.936700  0.134331  0.678820
5           0.349497  0.036978  0.228787  0.043461  0.899230  0.369351

e que os níveis podem ser passados para colunas e indices ao mesmo tempo:

In [26]:
lin = pd.MultiIndex.from_product([['manhã', 'noite'], [1, 0]], names=['momento', 'solução'])
col = pd.MultiIndex.from_product([['ontem', 'hoje', 'amanhã'], ['a', 'b']], names=['dia', 'ocorrência'])

df = pd.DataFrame(np.random.random((4, 6)), columns=col, index=lin)
df

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.092291  0.220387  0.826558  0.220524  0.752449  0.185577
        0        0.056871  0.281663  0.329854  0.067071  0.149826  0.029238
noite   1        0.235174  0.660855  0.226458  0.503577  0.444814  0.106559
        0        0.897410  0.621478  0.409184  0.911634  0.762789  0.946466

#### indexing e slicing em dataframes
---

para indexing, é importante lembrar que sempre as colunas vêm primeiro; enquanto que para slicing, quemvem primeiro são as linhas.

nestes casos, os processos de indexing e slicing ocorrem nos seguintes moldes:

In [27]:
df['hoje']

ocorrência              a         b
momento solução                    
manhã   1        0.826558  0.220524
        0        0.329854  0.067071
noite   1        0.226458  0.503577
        0        0.409184  0.911634

In [28]:
df['hoje', 'a']

momento  solução
manhã    1          0.826558
         0          0.329854
noite    1          0.226458
         0          0.409184
Name: (hoje, a), dtype: float64

In [35]:
df['ontem', 'b']['manhã', 0]

0.2816629519250765

In [31]:
df['amanhã', 'a'][:, 1]

momento
manhã    0.752449
noite    0.444814
Name: (amanhã, a), dtype: float64

In [37]:
df.loc[:, ('ontem', 'a')]

momento  solução
manhã    1          0.092291
         0          0.056871
noite    1          0.235174
         0          0.897410
Name: (ontem, a), dtype: float64

a atributo `.indexSlice` ajuda, ainda, no slicing, já que, no geral, esta estrutura nãoo suporta tão processo:

In [40]:
idx = pd.IndexSlice
df.loc[idx[:, 0], idx['amanhã', :]]

dia                amanhã          
ocorrência              a         b
momento solução                    
manhã   0        0.149826  0.029238
noite   0        0.762789  0.946466

este atributo só pode ser usado com o atributo `.loc`

In [41]:
df.loc[idx[:, 1], idx[:, :]]

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.092291  0.220387  0.826558  0.220524  0.752449  0.185577
noite   1        0.235174  0.660855  0.226458  0.503577  0.444814  0.106559

#### reset_index
---

este método reformula o dataframe ou series, tornando os níveis em simples colunas separadas:

In [52]:
df_flat = df.reset_index()
df_flat

dia        momento solução     ontem                hoje              amanhã  \
ocorrência                         a         b         a         b         a   
0            manhã       1  0.092291  0.220387  0.826558  0.220524  0.752449   
1            manhã       0  0.056871  0.281663  0.329854  0.067071  0.149826   
2            noite       1  0.235174  0.660855  0.226458  0.503577  0.444814   
3            noite       0  0.897410  0.621478  0.409184  0.911634  0.762789   

dia                   
ocorrência         b  
0           0.185577  
1           0.029238  
2           0.106559  
3           0.946466

ainda recebe o parâmetro `name=` que nomea a(s) nova(s) coluna(s):

#### set_index()
---

esta função faz o inverso da anterior, organizando series e dataframes com multiindexing:

In [57]:
df_flat.set_index(['momento', 'solução'])

dia                 ontem                hoje              amanhã          
ocorrência              a         b         a         b         a         b
momento solução                                                            
manhã   1        0.092291  0.220387  0.826558  0.220524  0.752449  0.185577
        0        0.056871  0.281663  0.329854  0.067071  0.149826  0.029238
noite   1        0.235174  0.660855  0.226458  0.503577  0.444814  0.106559
        0        0.897410  0.621478  0.409184  0.911634  0.762789  0.946466

observe que é necessário passar o nome para os índices.